Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install boto3

In [2]:
import boto3
import sys
import config


In [ ]:
aws_access_key_id=config.aws_access_key_id
aws_secret_access_key=config.aws_secret_access_key
session=boto3.Session(aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)



In [ ]:
#aBRINDO CLINETE
client=session.client('rekognition',region_name='us-east-2')

In [ ]:
#Criando um coleção
response=client.create_collection(CollectionId='user-faces')


ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateCollection operation: The collection id: user-faces already exists

In [ ]:
#Criando nova pessoas
response=client.create_user(CollectionId='user-faces',UserId='nicolas-cage')

InvalidParameterException: An error occurred (InvalidParameterException) when calling the CreateUser operation: Request has invalid parameters

In [ ]:
##nome do bucket
bucket_name='fiap-cognitive-enviroments'
#Lista de face Ids
faceIds=[]
for n in range(1,4):
  if n==1:
    image_type='jpg'
  else:
    image_type='png'
  file_name=f'nic{n}.{image_type}'
  response=client.index_faces(CollectionId='user-faces',
                               Image={
                                   'S3Object':{
                                       'Bucket':bucket_name,
                                       'Name':file_name
                                       }
                                 }

                               )
  faceIds.append(response['FaceRecords'][0]['Face']['FaceId'])
faceIds

['287842e2-b689-49af-bb06-8120c22fcc91',
 '0b7b78da-c9b5-41ac-950d-adb0397691cb',
 'efea902b-bb70-4909-8cfd-a0c5f59ad6c8']

In [ ]:
response=client.associate_faces(
    CollectionId='user-faces',
    UserId='nicolas-cage',
    FaceIds=faceIds
)

In [ ]:
response=client.search_faces_by_image(
     Image={
                                   'S3Object':{
                                       'Bucket':bucket_name,
                                       'Name':'nic4.png'
                                   }
     },
    CollectionId='user-faces'
)

In [ ]:
response

{'SearchedFaceBoundingBox': {'Width': 0.4799571931362152,
  'Height': 0.45555803179740906,
  'Left': 0.24742986261844635,
  'Top': 0.17509520053863525},
 'SearchedFaceConfidence': 99.9996566772461,
 'FaceMatches': [{'Similarity': 99.99962615966797,
   'Face': {'FaceId': 'efea902b-bb70-4909-8cfd-a0c5f59ad6c8',
    'BoundingBox': {'Width': 0.4293079972267151,
     'Height': 0.4860459864139557,
     'Left': 0.27021700143814087,
     'Top': 0.18169599771499634},
    'ImageId': '57e03d4f-9006-34e0-abd3-dfd35d5f4c9e',
    'Confidence': 99.99930572509766,
    'IndexFacesModelVersion': '7.0'}},
  {'Similarity': 99.99945831298828,
   'Face': {'FaceId': '287842e2-b689-49af-bb06-8120c22fcc91',
    'BoundingBox': {'Width': 0.5695030093193054,
     'Height': 0.5277789831161499,
     'Left': 0.22421999275684357,
     'Top': 0.18140099942684174},
    'ImageId': '4db2a529-d34d-37a0-a181-75aa9dd2a3c2',
    'Confidence': 99.99949645996094,
    'IndexFacesModelVersion': '7.0'}},
  {'Similarity': 99.99595

In [ ]:
face_id_max_similarity=response['FaceMatches'][0]['Face']['FaceId']

In [ ]:
response=client.search_users(
    FaceId=face_id_max_similarity,
    CollectionId='user-faces'
)

In [ ]:
response

{'UserMatches': [{'Similarity': 99.99998474121094,
   'User': {'UserId': 'nicolas-cage', 'UserStatus': 'ACTIVE'}}],
 'FaceModelVersion': '7',
 'SearchedFace': {'FaceId': 'efea902b-bb70-4909-8cfd-a0c5f59ad6c8'},
 'ResponseMetadata': {'RequestId': 'bda37109-6a46-4fab-a5ca-e895ebe92acf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bda37109-6a46-4fab-a5ca-e895ebe92acf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '193',
   'date': 'Sun, 24 Mar 2024 19:25:09 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# prompt: response

print(response)


{'UserMatches': [{'Similarity': 99.99998474121094, 'User': {'UserId': 'nicolas-cage', 'UserStatus': 'ACTIVE'}}], 'FaceModelVersion': '7', 'SearchedFace': {'FaceId': 'efea902b-bb70-4909-8cfd-a0c5f59ad6c8'}, 'ResponseMetadata': {'RequestId': 'bda37109-6a46-4fab-a5ca-e895ebe92acf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bda37109-6a46-4fab-a5ca-e895ebe92acf', 'content-type': 'application/x-amz-json-1.1', 'content-length': '193', 'date': 'Sun, 24 Mar 2024 19:25:09 GMT'}, 'RetryAttempts': 0}}


In [ ]:
user_name=response['UserMatches'][0]['User']['UserId']

In [ ]:
user_name

'nicolas-cage'

In [ ]:
def connect_aws():
  aws_access_key_id=config.aws_access_key_id
  aws_secret_access_key=config.aws_secret_access_key
  session=boto3.Session(aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)
  return session


def recognize_user(image):
  """
  Retorna nome do usuário de uma imagem
  Variavel
  image: imagem em bytes
  retorna str
  """
  #Conecta com AWS
  session=connect_aws()
  #aBRINDO CLINETE
  client=session.client('rekognition',region_name='us-east-2')
  #Obtendo dados da imagem
  response=client.search_faces_by_image(
     Image={'Bytes': image
     },
    CollectionId='user-faces'
  )
  #Obtendo face_id com maior similaridade
  face_id_max_similarity=response['FaceMatches'][0]['Face']['FaceId']
  #Obtendo User de com maior similaridade
  response=client.search_users(
    FaceId=face_id_max_similarity,
    CollectionId='user-faces'
  )
  #Retorna nome do usuário
  return response['UserMatches'][0]['User']['UserId']

def count_faces(img):
  """
  Função que conta numero de pessoas
  variavel
  img - Imagem em bytes
  retorna número de imagens
  """
  #Conecta com AWS
  session=connect_aws()
  #aBRINDO CLINETE
  client=session.client('rekognition',region_name='us-east-2')
  #Criando requisição
  response=client.detect_faces(
      Image={'Bytes':img}
      # Attributes=['All']
  )
  #Contando numero de pessoas
  n= len(n['FaceDetails'])
  return n



In [3]:
path='C:\\Users\\alber\\Documents\\Projetos - Dados\\fiap_cognitive_enviroment_liveness\\notebooks\\Cadastro\\imgs'
with open(f'{path}/nic4.png', "rb") as file:
  img_file = file.read()
  bytes_file_target = bytearray(img_file)

In [ ]:
recognize_user(bytes_file_target)

IndexError: list index out of range

In [4]:
with open(f'{path}/selfie_oscar.jpg', "rb") as file:
  img_file = file.read()
  bytes_file_target = bytearray(img_file)
n=count_faces(bytes_file_target)

NameError: name 'count_faces' is not defined